#### Analyte classification using CNN
#### Edgar Acuna
#### Reviewed  June 2021

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_iris
from numpy import unique
import pandas as pd
import numpy as np
import time
import keras

In [2]:
df1=pd.read_csv("c://onr2020/NRLset1_part1.csv",header=None)
df2=pd.read_csv("c://onr2020/NRLset1_part2.csv",header=None)
df3=pd.read_csv("c://onr2020/NRLset1_part3.csv",header=None)
df4=pd.read_csv("c://onr2020/NRLset1_part4.csv",header=None)
df5=pd.read_csv("c://onr2020/NRLset1_part5.csv",header=None)
df6=pd.read_csv("c://onr2020/NRLset1_part6.csv",header=None)
df7=pd.read_csv("c://onr2020/NRLset1_part7.csv",header=None)
df8=pd.read_csv("c://onr2020/NRLset1_part8.csv",header=None)
y=pd.read_csv("c://onr2020/labels.csv",header=None)
ys=pd.read_csv("c://onr2020/substrateIDs.csv",header=None)
subs=pd.read_csv("c://onr2020/substrates.csv",header=None)
dfset1=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8],ignore_index=True)
print('Size of the dataframe: {}'.format(dfset1.shape))

Size of the dataframe: (18000, 1701)


In [3]:
dfset2=dfset1.copy()
dfset2['Analyte']=y
dfset2['substrate']=ys
dfsub1=dfset2[dfset2['substrate']==1]
dfsub2=dfset2[dfset2['substrate']==2]
dfsub3=dfset2[dfset2['substrate']==3]
dfsub4=dfset2[dfset2['substrate']==4]
dfsub5=dfset2[dfset2['substrate']==5]
dfsub6=dfset2[dfset2['substrate']==6]
dfsub7=dfset2[dfset2['substrate']==7]
dfsub8=dfset2[dfset2['substrate']==8]
dfsub9=dfset2[dfset2['substrate']==9]

In [4]:
mdfsub1=dfsub1.iloc[:,0:1701]
mdfsub2=dfsub2.iloc[:,0:1701]
mdfsub3=dfsub3.iloc[:,0:1701]
mdfsub4=dfsub4.iloc[:,0:1701]
mdfsub5=dfsub5.iloc[:,0:1701]
mdfsub6=dfsub6.iloc[:,0:1701]
mdfsub7=dfsub7.iloc[:,0:1701]
mdfsub8=dfsub8.iloc[:,0:1701]
mdfsub9=dfsub9.iloc[:,0:1701]

In [5]:
y1=subs.loc[0,:]
y2=subs.loc[1,:]
y3=subs.loc[2,:]
y4=subs.loc[3,:]
y5=subs.loc[4,:]
y6=subs.loc[5,:]
y7=subs.loc[6,:]
y8=subs.loc[7,:]
y9=subs.loc[8,:]

### Centering

In [6]:
modsub1=mdfsub1.apply(lambda x : x -(np.sum(np.array(x)*np.array(y1))/np.sum(np.array(y1)*np.array(y1)))*y1,axis=1)
modsub2=mdfsub2.apply(lambda x : x -(np.sum(np.array(x)*np.array(y2))/np.sum(np.array(y2)*np.array(y2)))*y2,axis=1)
modsub3=mdfsub3.apply(lambda x : x -(np.sum(np.array(x)*np.array(y3))/np.sum(np.array(y3)*np.array(y3)))*y3,axis=1)
modsub4=mdfsub4.apply(lambda x : x -(np.sum(np.array(x)*np.array(y4))/np.sum(np.array(y4)*np.array(y4)))*y4,axis=1)
modsub5=mdfsub5.apply(lambda x : x -(np.sum(np.array(x)*np.array(y5))/np.sum(np.array(y5)*np.array(y5)))*y5,axis=1)
modsub6=mdfsub6.apply(lambda x : x -(np.sum(np.array(x)*np.array(y6))/np.sum(np.array(y6)*np.array(y6)))*y6,axis=1)
modsub7=mdfsub7.apply(lambda x : x -(np.sum(np.array(x)*np.array(y7))/np.sum(np.array(y7)*np.array(y7)))*y7,axis=1)
modsub8=mdfsub8.apply(lambda x : x -(np.sum(np.array(x)*np.array(y8))/np.sum(np.array(y8)*np.array(y8)))*y8,axis=1)
modsub9=mdfsub9.apply(lambda x : x -(np.sum(np.array(x)*np.array(y9))/np.sum(np.array(y9)*np.array(y9)))*y9,axis=1)

In [7]:
subdf=[modsub1,modsub2,modsub3,modsub4,modsub5,modsub6,modsub7,modsub8,modsub9]
cent_subs=pd.concat(subdf)
cent_subs.head()

,0,1,2,3,4,5,6,7,8,9,...,1691,1692,1693,1694,1695,1696,1697,1698,1699,1700
4,0.002296,0.002603,0.002368,0.003507,0.003637,0.003537,0.004850,0.005059,0.004744,0.006244,...,-0.001858,-0.002531,-0.002143,-0.004422,-0.003123,-0.000884,-0.001225,-0.001406,-0.002792,-0.002432
5,-0.005471,0.003865,0.000351,-0.000060,-0.000258,-0.000582,0.002557,0.003072,0.000961,0.003941,...,-0.004245,-0.001915,-0.005149,-0.001717,-0.003814,-0.003100,-0.000355,-0.000540,-0.005481,-0.000857
8,-0.002496,0.016867,0.014204,0.005327,0.003649,0.006143,-0.015160,0.004201,0.003791,0.004599,...,0.002574,-0.018040,-0.018488,-0.006529,0.009548,0.004694,0.000399,0.018158,0.016369,0.007582
28,0.001313,0.001443,0.001536,0.001612,0.001687,0.001768,0.001866,0.001979,0.002079,0.002155,...,-0.004576,-0.004550,-0.004524,-0.004502,-0.004476,-0.004446,-0.004411,-0.004373,-0.004332,-0.004301
46,-0.001097,0.000040,0.001125,0.002056,0.001685,0.002006,0.000228,0.001179,0.001835,0.002005,...,-0.004655,-0.006438,-0.005102,-0.005563,-0.004804,-0.005517,-0.004411,-0.005738,-0.005031,-0.005584


### Normalization

In [8]:
b=cent_subs.iloc[:,0:1701]
b.head()

,0,1,2,3,4,5,6,7,8,9,...,1691,1692,1693,1694,1695,1696,1697,1698,1699,1700
4,0.002296,0.002603,0.002368,0.003507,0.003637,0.003537,0.004850,0.005059,0.004744,0.006244,...,-0.001858,-0.002531,-0.002143,-0.004422,-0.003123,-0.000884,-0.001225,-0.001406,-0.002792,-0.002432
5,-0.005471,0.003865,0.000351,-0.000060,-0.000258,-0.000582,0.002557,0.003072,0.000961,0.003941,...,-0.004245,-0.001915,-0.005149,-0.001717,-0.003814,-0.003100,-0.000355,-0.000540,-0.005481,-0.000857
8,-0.002496,0.016867,0.014204,0.005327,0.003649,0.006143,-0.015160,0.004201,0.003791,0.004599,...,0.002574,-0.018040,-0.018488,-0.006529,0.009548,0.004694,0.000399,0.018158,0.016369,0.007582
28,0.001313,0.001443,0.001536,0.001612,0.001687,0.001768,0.001866,0.001979,0.002079,0.002155,...,-0.004576,-0.004550,-0.004524,-0.004502,-0.004476,-0.004446,-0.004411,-0.004373,-0.004332,-0.004301
46,-0.001097,0.000040,0.001125,0.002056,0.001685,0.002006,0.000228,0.001179,0.001835,0.002005,...,-0.004655,-0.006438,-0.005102,-0.005563,-0.004804,-0.005517,-0.004411,-0.005738,-0.005031,-0.005584


In [9]:
#normalize(b)
b1=b.apply(lambda x: x/(x**2).sum()**.5, axis=1)
b1.shape

(18000, 1701)

In [10]:
a1=np.array(dfsub1['Analyte'].values.tolist())
a2=np.array(dfsub2['Analyte'].values.tolist())
a3=np.array(dfsub3['Analyte'].values.tolist())
a4=np.array(dfsub4['Analyte'].values.tolist())
a5=np.array(dfsub5['Analyte'].values.tolist())
a6=np.array(dfsub6['Analyte'].values.tolist())
a7=np.array(dfsub7['Analyte'].values.tolist())
a8=np.array(dfsub8['Analyte'].values.tolist())
a9=np.array(dfsub9['Analyte'].values.tolist())
y=np.concatenate((a1,a2,a3,a4,a5,a6,a7,a8,a9),axis=None)
#print(len(y))

In [11]:
x=b1.iloc[:,0:1701].to_numpy()
x = x.reshape(x.shape[0], x.shape[1], 1)
print(x.shape)
y=y-1
print(unique(y))
print(unique(y).sum())

xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.2)


(18000, 1701, 1)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39]
780


In [12]:
model = Sequential()
model.add(Conv1D(64, 2, activation="relu", input_shape=(1701,1)))
model.add(Dense(16, activation="relu"))
model.add(MaxPooling1D())
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(40, activation = 'softmax'))
start_time = time.time()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
model.summary()
model.fit(xtrain, ytrain, batch_size=64,epochs=50,  validation_split=.2, verbose=1)

acc = model.evaluate(xtrain, ytrain)
print("Loss:", acc[0], " Accuracy:", acc[1])

pred = model.predict(xtest)
pred_y = pred.argmax(axis=-1)
print("--- %s seconds ---" % (time.time() - start_time))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1700, 64)          192       
_________________________________________________________________
dense (Dense)                (None, 1700, 16)          1040      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 850, 16)           0         
_________________________________________________________________
dropout (Dropout)            (None, 850, 16)           0         
_________________________________________________________________
flatten (Flatten)            (None, 13600)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 40)                544040    
Total params: 545,272
Trainable params: 545,272
Non-trainable params: 0
__________________________________________________

180/180 [==============================] - 11s 63ms/step - loss: 1.2696e-04 - accuracy: 1.0000 - val_loss: 0.2030 - val_accuracy: 0.9601
Epoch 50/50
450/450 [==============================] - 6s 14ms/step - loss: 0.0419 - accuracy: 0.9922
Loss: 0.041927747428417206  Accuracy: 0.992222249507904
--- 586.5150573253632 seconds ---


In [13]:
# Test the model after training
test_results = model.evaluate(xtest, ytest, verbose=1)
print(test_results)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')
print("--- %s seconds ---" % (time.time() - start_time))

113/113 [==============================] - 2s 15ms/step - loss: 0.1883 - accuracy: 0.9617
[0.1883438378572464, 0.9616666436195374]
Test results - Loss: 0.1883438378572464 - Accuracy: 0.9616666436195374%
--- 588.6291143894196 seconds ---


In [14]:
#Computing F1-score
import sklearn
train_features = np.array(xtrain)
test_features = np.array(xtest)
train_labels=np.array(ytrain)
test_labels=np.array(ytest)
train_predictions_baseline = model.predict_classes(train_features, batch_size=64)
#train_predictions_baseline =np.argmax(model.predict(train_features, batch_size=64)>0.5).astype("int32")
f1_train=sklearn.metrics.f1_score(ytrain, train_predictions_baseline, average="weighted")
test_predictions_baseline = model.predict_classes(test_features, batch_size=64)
f1_test=sklearn.metrics.f1_score(ytest, test_predictions_baseline, average="weighted")
print('f1_scores in training set',f1_train,'f1_scores in testing set',f1_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
f1_scores in training set 0.9922333573503361 f1_scores in testing set 0.9619282032136905


### Computing metrics per analyte

In [28]:
labels1=[12,33,36]
import time
from sklearn.metrics import confusion_matrix
start_time=time.time()
for j in labels1:
    yclass=y.copy()
    #unique_elements, counts_elements = np.unique(yclass, return_counts=True)
    #print("Frequency of unique values of the said array:") 
    #print(np.asarray((unique_elements, counts_elements)))
    #print(yclass)
    yclass[yclass!=j]=0 
    yclass[yclass==j]=1
    xtrain, xtest, yclasstrain, yclasstest = train_test_split(x,yclass,test_size=0.2,random_state=0)
    #print(xtrain.shape)
    #print(xtest.shape)
    #ytrain = to_categorical(yclass_train, num_classes)
    #ytest = to_categorical(yclass_test, num_classes)
    model = Sequential()
    model.add(Conv1D(64, 2, activation="relu", input_shape=(1701,1)))
    model.add(Dense(16, activation="relu"))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(1, activation = 'sigmoid'))
    start_time = time.time()
    #model.compile(loss = 'sparse_categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    model.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    model.summary()
    model.fit(xtrain, yclasstrain, batch_size=64,epochs=20,  validation_split=.2, verbose=1)
    acc = model.evaluate(xtrain, yclasstrain)
    print("Loss:", acc[0], " Accuracy:", acc[1])
    pred = model.predict_classes(xtest)
    #pred_y = pred.argmax(axis=-1)
    #cm = confusion_matrix(ytest, pred)
    #print(cm)
    print("--- %s seconds ---" % (time.time() - start_time))
    #pred = model.predict(xtest)
    #pred_y = pred.argmax(axis=-1)
    #cm = confusion_matrix(ytest, pred)
    #print(pred)
    #Computing F1-score
    import sklearn
    train_features = np.array(xtrain)
    test_features = np.array(xtest)
    train_labels=np.array(yclasstrain)
    test_labels=np.array(yclasstest)
    #print("obser",np.count_nonzero(yclasstest))
    train_predictions_baseline = model.predict_classes(train_features, batch_size=64)
    f1_train=sklearn.metrics.f1_score(yclasstrain, train_predictions_baseline, average="weighted")
    test_predictions_baseline = model.predict_classes(test_features, batch_size=64)
    #print("pred",np.count_nonzero(test_predictions_baseline))
    cm = confusion_matrix(yclasstest, test_predictions_baseline)
    print("The confusion matrix\n",cm)
    f1_test=sklearn.metrics.f1_score(yclasstest, test_predictions_baseline, average="weighted")
    print("Analyte is",j+1)
    print('f1_scores in training set',f1_train,'f1_scores in testing set',f1_test)


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1700, 64)          192       
_________________________________________________________________
dense_2 (Dense)              (None, 1700, 16)          1040      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 850, 16)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 850, 16)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 13600)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 13601     
Total params: 14,833
Trainable params: 14,833
Non-trainable params: 0
__________________________________________________

Epoch 20/20
450/450 [==============================] - 3s 7ms/step - loss: 0.0053 - accuracy: 0.9991
Loss: 0.005255263298749924  Accuracy: 0.9990972280502319
--- 133.70451760292053 seconds ---
The confusion matrix
 [[3505    4]
 [  11   80]]
Analyte is 34
f1_scores in training set 0.9990966087434614 f1_scores in testing set 0.9957520759193357
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 1700, 64)          192       
_________________________________________________________________
dense_6 (Dense)              (None, 1700, 16)          1040      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 850, 16)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 850, 16)           0         
_______________________________________

###  PCA before performing analytes classification

In [42]:
from sklearn.decomposition import PCA
dfset3=b1.copy()
pca = PCA(n_components=30)
pca_result = pca.fit_transform(dfset3)

In [43]:
x=pca_result
x = x.reshape(x.shape[0], x.shape[1], 1)
xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.2)
print(x.shape)

(18000, 30, 1)


In [45]:
model = Sequential()
model.add(Conv1D(16, 2, activation="relu", input_shape=(30,1)))
model.add(Dense(8, activation="relu"))
model.add(MaxPooling1D())
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(40, activation = 'softmax'))
start_time = time.time()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
model.summary()
model.fit(xtrain, ytrain, batch_size=64,epochs=50,  validation_split=.2, verbose=1)

acc = model.evaluate(xtrain, ytrain)
print("Loss:", acc[0], " Accuracy:", acc[1])

pred = model.predict(xtest)
pred_y = pred.argmax(axis=-1)
print("--- %s seconds ---" % (time.time() - start_time))

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 29, 16)            48        
_________________________________________________________________
dense_16 (Dense)             (None, 29, 8)             136       
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 14, 8)             0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 14, 8)             0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 112)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 40)                4520      
Total params: 4,704
Trainable params: 4,704
Non-trainable params: 0
____________________________________________________

In [46]:
# Test the model after training
test_results = model.evaluate(xtest, ytest, verbose=1)
print(test_results)
print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]}%')

113/113 [==============================] - 0s 939us/step - loss: 0.1648 - accuracy: 0.9589
[0.16482645273208618, 0.9588888883590698]
Test results - Loss: 0.16482645273208618 - Accuracy: 0.9588888883590698%


In [48]:
#Computing F1-score
import sklearn
train_features = np.array(xtrain)
test_features = np.array(xtest)
train_labels=np.array(ytrain)
test_labels=np.array(ytest)
train_predictions_baseline = model.predict_classes(train_features, batch_size=64)
f1_train=sklearn.metrics.f1_score(ytrain, train_predictions_baseline, average="weighted")
test_predictions_baseline = model.predict_classes(test_features, batch_size=64)
f1_test=sklearn.metrics.f1_score(ytest, test_predictions_baseline, average="weighted")
print('f1_scores in training set',f1_train,'f1_scores in testing set',f1_test)

f1_scores in training set 0.9648452282775338 f1_scores in testing set 0.9588181401400551


In [ ]:
labels1=[0,1,2,3,4,5]
import time
start_time=time.time()
for j in labels1:
    yclass=y.copy()
    yclass[yclass!=j]=0 
    yclass[yclass==j]=1
    xtrain, xtest, yclasstrain, yclasstest = train_test_split(x,yclass,test_size=0.2,random_state=0)
    #print('Train dimension:');print(X_train.shape)
    #print('Test dimension:');print(X_test.shape)
    #ytrain = to_categorical(yclass_train, num_classes)
    #ytest = to_categorical(yclass_test, num_classes)
    model = Sequential()
    model.add(Conv1D(16, 2, activation="relu", input_shape=(50,1)))
    model.add(Dense(8, activation="relu"))
    model.add(Dropout(0.25))
    model.add(MaxPooling1D())
    model.add(Flatten())
    model.add(Dense(1, activation = 'sigmoid'))
    start_time = time.time()
    #model.compile(loss = 'sparse_categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    model.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    #model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[keras.metrics.BinaryAccuracy(name="accuracy")])
    model.summary()
    model.fit(xtrain, yclasstrain, batch_size=64,epochs=100,  validation_split=.2, verbose=1)
    acc = model.evaluate(xtrain, yclasstrain)
    print("Loss:", acc[0], " Accuracy:", acc[1])
    pred = model.predict(xtest)
    pred_y = pred.argmax(axis=-1)
    print("--- %s seconds ---" % (time.time() - start_time))
    #Computing F1-score
    import sklearn
    train_features = np.array(xtrain)
    test_features = np.array(xtest)
    train_labels=np.array(yclasstrain)
    test_labels=np.array(yclasstest)
    train_predictions_baseline = model.predict_classes(train_features, batch_size=64)
    f1_train=sklearn.metrics.f1_score(yclasstrain, train_predictions_baseline, average="weighted")
    test_predictions_baseline = model.predict_classes(test_features, batch_size=64)
    f1_test=sklearn.metrics.f1_score(yclasstest, test_predictions_baseline, average="weighted")
    print("Analyte is",j+1)
    print('f1_scores in training set',f1_train,'f1_scores in testing set',f1_test)
    